### Pickle verification results

In [1]:
import pickle
import evaluation

In [2]:
# with open("nagini_examples/verif_result.pkl", "rb") as f:
#     verif_result = pickle.load(f)
# print(verif_result)
ev = evaluation.Evaluation("config_with_sorting.json")
verif_result:dict[str,str] = {}
examples = ev.data.get_list_of_examples("list")
for example in examples:
    program_snippet = ev.data.get_example("list", example, "unverified")
    result = ev._verify_program_snippet("list", program_snippet)
    verif_result[example] = str(result)
with open("nagini_examples/verif_result_sorting.pkl", "wb") as f:
    pickle.dump(verif_result, f)

response ['', 'Verification successful', 'Verification took 2.53 seconds.']
response ['', 'Verification failed', 'Errors:', 'Conditional statement might fail. There might be insufficient permission to access head.next. (tmp.py@23.7)', 'Verification took 2.02 seconds.']
response ['', 'Verification failed', 'Errors:', 'Conditional statement might fail. There might be insufficient permission to access head.val. (tmp.py@25.7)', 'Verification took 2.00 seconds.']
response ['', 'Verification failed', 'Errors:', 'Conditional statement might fail. There might be insufficient permission to access ptr.val. (tmp.py@25.11)', 'Verification took 2.01 seconds.']
response ['', 'Verification failed', 'Errors:', 'Conditional statement might fail. There might be insufficient permission to access head.val. (tmp.py@23.7)', 'Verification took 1.89 seconds.']
response ['', 'Verification failed', 'Errors:', 'Method call might fail. There might be insufficient permission to access head1.next. (tmp.py@27.17)', 

In [3]:
with open("nagini_examples/verif_result_sorting.pkl", "rb") as f:
    verif_result = pickle.load(f)
print(verif_result)


{'prepend': 'Verification successful', 'append': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access head.next. at line 3.7', 'find': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access head.val. at line 5.7', 'find_iter': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access ptr.val. at line 5.11', 'remove': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access head.val. at line 3.7', 'join_lists': 'Verification failed: Method call might fail. There might be insufficient permission to access head1.next. at line 7.17', 'reverse': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access head.next. at line 3.7', 'merge': 'Verification failed: Conditional statement might fail. There might be insufficient permission to access head1.val. a